In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install geopandas pandas
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 50.0 MB/s eta 0:00:00


In [ ]:
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import numpy as np
from shapely.geometry import mapping
import matplotlib.pyplot as plt

In [ ]:
parks_file = '/content/drive/MyDrive/Capstone/IMPORTANT_DATA/filtered_park_df.geojson'  # 替换为你文件的路径
parks_gdf = gpd.read_file(parks_file)

accessbility_index_file = "/content/drive/MyDrive/Capstone/IMPORTANT_DATA/accessibility_2sfca.geojson"
accessibility_gdf = gpd.read_file(accessbility_index_file )

In [ ]:
accessibility_gdf

,id,population,2sfca_40_Shape_Area,geometry
0,19642,80.844990,2.267050,POINT (13518650 3664926.845)
1,19643,83.608226,2.112203,POINT (13518750 3664926.845)
2,19644,69.003618,3.266251,POINT (13518850 3664926.845)
3,19875,78.576787,2.103589,POINT (13518450 3664809.878)
4,19876,76.828783,2.591295,POINT (13518550 3664809.878)
...,...,...,...,...
2016,36377,64.789033,0.110101,POINT (13521850 3657676.968)
2017,36378,60.187111,0.195637,POINT (13521950 3657676.968)
2018,36379,58.169425,0.119731,POINT (13522050 3657676.968)
2019,36380,68.189320,0.237274,POINT (13522150 3657676.968)


In [ ]:
parks_gdf

,OBJECTID,osm_id,code,fclass,name,Shape_Leng,Shape_Area,geometry
0,56,39763744,7202,park,人民公园,0.013504,9.854620e-06,"MULTIPOLYGON (((121.47083 31.23446, 121.47125 ..."
1,57,39961770,7202,park,复兴公园,0.011324,6.920550e-06,"MULTIPOLYGON (((121.46457 31.22042, 121.46464 ..."
2,129,50258395,7202,park,大观园绿地,0.004629,9.350000e-07,"MULTIPOLYGON (((121.46866 31.23965, 121.46744 ..."
3,246,58413664,7202,park,None,0.004009,4.514200e-07,"MULTIPOLYGON (((121.47952 31.19812, 121.47822 ..."
4,247,58413720,7202,park,None,0.005192,9.922400e-07,"MULTIPOLYGON (((121.48059 31.19819, 121.48047 ..."
...,...,...,...,...,...,...,...,...
105,5513,477138725,7202,park,None,0.001543,1.101900e-07,"MULTIPOLYGON (((121.44744 31.21099, 121.44719 ..."
106,5514,477138726,7202,park,None,0.001404,1.157900e-07,"MULTIPOLYGON (((121.44357 31.21067, 121.44327 ..."
107,6469,571607270,7202,park,None,0.002431,2.940200e-07,"MULTIPOLYGON (((121.44621 31.19872, 121.4466 3..."
108,9370,870546902,7202,park,None,0.008064,1.239510e-06,"MULTIPOLYGON (((121.45771 31.17676, 121.45772 ..."


In [ ]:
# 将公园数据转换为投影坐标系（以米为单位，方便创建缓冲区）
parks_gdf = parks_gdf.to_crs(epsg=32650)

# 创建 500 米缓冲区
parks_gdf['buffer_3000m'] = parks_gdf.geometry.buffer(3000)

# 查看缓冲区
print(parks_gdf[['OBJECTID', 'buffer_3000m']].head())
parks_gdf

   OBJECTID                                       buffer_3000m
0        56  POLYGON ((923402.298 3461957.981, 923275.215 3...
1        57  POLYGON ((922682.566 3461001.226, 922595.943 3...
2       129  POLYGON ((928467.94 3463543.746, 928328.472 34...
3       246  POLYGON ((927224.859 3457227.527, 927138.467 3...
4       247  POLYGON ((927411.528 3457247.844, 927246.351 3...


,OBJECTID,osm_id,code,fclass,name,Shape_Leng,Shape_Area,geometry,buffer_3000m
0,56,39763744,7202,park,人民公园,0.013504,9.854620e-06,"MULTIPOLYGON (((925966.322 3464215.256, 926008...","POLYGON ((923402.298 3461957.981, 923275.215 3..."
1,57,39961770,7202,park,复兴公园,0.011324,6.920550e-06,"MULTIPOLYGON (((925432.001 3462632.445, 925439...","POLYGON ((922682.566 3461001.226, 922595.943 3..."
2,129,50258395,7202,park,大观园绿地,0.004629,9.350000e-07,"MULTIPOLYGON (((925735.847 3464782.966, 925621...","POLYGON ((928467.94 3463543.746, 928328.472 34..."
3,246,58413664,7202,park,None,0.004009,4.514200e-07,"MULTIPOLYGON (((926958.678 3460215.929, 926835...","POLYGON ((927224.859 3457227.527, 927138.467 3..."
4,247,58413720,7202,park,None,0.005192,9.922400e-07,"MULTIPOLYGON (((927060.441 3460227.229, 927048...","POLYGON ((927411.528 3457247.844, 927246.351 3..."
...,...,...,...,...,...,...,...,...,...
105,5513,477138725,7202,park,None,0.001543,1.101900e-07,"MULTIPOLYGON (((923840.196 3461519.881, 923818...","POLYGON ((922716.033 3464301.293, 922999.546 3..."
106,5514,477138726,7202,park,None,0.001404,1.157900e-07,"MULTIPOLYGON (((923472.148 3461469.703, 923444...","POLYGON ((926192.413 3462734.676, 926304.195 3..."
107,6469,571607270,7202,park,None,0.002431,2.940200e-07,"MULTIPOLYGON (((923777.762 3460153.043, 923818...","POLYGON ((922548.172 3462889.484, 922813.689 3..."
108,9370,870546902,7202,park,None,0.008064,1.239510e-06,"MULTIPOLYGON (((924973.594 3457760.709, 924975...","POLYGON ((921999.656 3457176.756, 921987.27 34..."


In [ ]:
accessibility_gdf =  accessibility_gdf.to_crs(epsg=32650)
print("Raster CRS:", accessibility_gdf.crs)

Raster CRS: EPSG:32650


In [ ]:
# 将 buffer_3000m 设置为公园的活动几何列
parks_gdf = parks_gdf.set_geometry('buffer_3000m')

# 确保 population_gdf 的 geometry 列是活动几何列
accessibility_gdf = accessibility_gdf.set_geometry('geometry')

In [ ]:
population_in_parks = gpd.sjoin(accessibility_gdf, parks_gdf[['OBJECTID', 'buffer_3000m']], how='inner', predicate='within')
# 计算每个公园的平均2sfca_40_Shape_Area
acc_2sfca = population_in_parks.groupby('OBJECTID')['2sfca_40_Shape_Area'].mean().reset_index()

# 将计算结果与原始的 park_df 合并，加入新列 acc_2sfca
park_df = parks_gdf.merge(acc_2sfca, on='OBJECTID', how='left')
park_df

,OBJECTID,osm_id,code,fclass,name,Shape_Leng,Shape_Area,geometry,buffer_3000m,2sfca_40_Shape_Area
0,56,39763744,7202,park,人民公园,0.013504,9.854620e-06,"MULTIPOLYGON (((925966.322 3464215.256, 926008...","POLYGON ((923402.298 3461957.981, 923275.215 3...",2.542174
1,57,39961770,7202,park,复兴公园,0.011324,6.920550e-06,"MULTIPOLYGON (((925432.001 3462632.445, 925439...","POLYGON ((922682.566 3461001.226, 922595.943 3...",2.372510
2,129,50258395,7202,park,大观园绿地,0.004629,9.350000e-07,"MULTIPOLYGON (((925735.847 3464782.966, 925621...","POLYGON ((928467.94 3463543.746, 928328.472 34...",2.679409
3,246,58413664,7202,park,None,0.004009,4.514200e-07,"MULTIPOLYGON (((926958.678 3460215.929, 926835...","POLYGON ((927224.859 3457227.527, 927138.467 3...",1.538381
4,247,58413720,7202,park,None,0.005192,9.922400e-07,"MULTIPOLYGON (((927060.441 3460227.229, 927048...","POLYGON ((927411.528 3457247.844, 927246.351 3...",1.544826
...,...,...,...,...,...,...,...,...,...,...
105,5513,477138725,7202,park,None,0.001543,1.101900e-07,"MULTIPOLYGON (((923840.196 3461519.881, 923818...","POLYGON ((922716.033 3464301.293, 922999.546 3...",2.260777
106,5514,477138726,7202,park,None,0.001404,1.157900e-07,"MULTIPOLYGON (((923472.148 3461469.703, 923444...","POLYGON ((926192.413 3462734.676, 926304.195 3...",2.254208
107,6469,571607270,7202,park,None,0.002431,2.940200e-07,"MULTIPOLYGON (((923777.762 3460153.043, 923818...","POLYGON ((922548.172 3462889.484, 922813.689 3...",1.803989
108,9370,870546902,7202,park,None,0.008064,1.239510e-06,"MULTIPOLYGON (((924973.594 3457760.709, 924975...","POLYGON ((921999.656 3457176.756, 921987.27 34...",1.766043


In [ ]:
print(park_df["2sfca_40_Shape_Area"].describe())

count    110.000000
mean       2.544040
std        0.979488
min        1.170018
25%        1.838489
50%        2.522410
75%        2.749812
max        6.663924
Name: 2sfca_40_Shape_Area, dtype: float64


In [ ]:
park_df['accessibility_label'] = (park_df['2sfca_40_Shape_Area'] >= 2.544040).astype(int)
park_df

,OBJECTID,osm_id,code,fclass,name,Shape_Leng,Shape_Area,geometry,buffer_3000m,2sfca_40_Shape_Area,accessibility_label
0,56,39763744,7202,park,人民公园,0.013504,9.854620e-06,"MULTIPOLYGON (((925966.322 3464215.256, 926008...","POLYGON ((923402.298 3461957.981, 923275.215 3...",2.542174,0
1,57,39961770,7202,park,复兴公园,0.011324,6.920550e-06,"MULTIPOLYGON (((925432.001 3462632.445, 925439...","POLYGON ((922682.566 3461001.226, 922595.943 3...",2.372510,0
2,129,50258395,7202,park,大观园绿地,0.004629,9.350000e-07,"MULTIPOLYGON (((925735.847 3464782.966, 925621...","POLYGON ((928467.94 3463543.746, 928328.472 34...",2.679409,1
3,246,58413664,7202,park,None,0.004009,4.514200e-07,"MULTIPOLYGON (((926958.678 3460215.929, 926835...","POLYGON ((927224.859 3457227.527, 927138.467 3...",1.538381,0
4,247,58413720,7202,park,None,0.005192,9.922400e-07,"MULTIPOLYGON (((927060.441 3460227.229, 927048...","POLYGON ((927411.528 3457247.844, 927246.351 3...",1.544826,0
...,...,...,...,...,...,...,...,...,...,...,...
105,5513,477138725,7202,park,None,0.001543,1.101900e-07,"MULTIPOLYGON (((923840.196 3461519.881, 923818...","POLYGON ((922716.033 3464301.293, 922999.546 3...",2.260777,0
106,5514,477138726,7202,park,None,0.001404,1.157900e-07,"MULTIPOLYGON (((923472.148 3461469.703, 923444...","POLYGON ((926192.413 3462734.676, 926304.195 3...",2.254208,0
107,6469,571607270,7202,park,None,0.002431,2.940200e-07,"MULTIPOLYGON (((923777.762 3460153.043, 923818...","POLYGON ((922548.172 3462889.484, 922813.689 3...",1.803989,0
108,9370,870546902,7202,park,None,0.008064,1.239510e-06,"MULTIPOLYGON (((924973.594 3457760.709, 924975...","POLYGON ((921999.656 3457176.756, 921987.27 34...",1.766043,0


In [ ]:
park_df.to_csv("/content/drive/MyDrive/Capstone/IMPORTANT_DATA/park_wth_label")
print("")